# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In this notebook, you will be performing more interactive visual analytics using `Folium`.

## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Import required Python packages

In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1. Mark all launch sites on a map


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
spacex_df=pd.read_csv('data/spacex_launch_geo.csv')
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [7]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Create a circle for the launch site with a popup label showing its name
    circle = folium.Circle([row['Lat'], row['Long']], radius=50, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker for the launch site with an icon showing its name
    marker = folium.Marker(
        [row['Lat'], row['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row["Launch Site"],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

## Task 2. Mark the success/failed launches for each site on the map


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()

Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [10]:
def marker_color(outcome):
    if outcome == 1:
        return 'green'
    else:
        return 'red'
        
spacex_df['marker_color'] = spacex_df['class'].apply(marker_color)
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [11]:
# Add markers for each launch with different colors based on success or failure
for index, row in spacex_df.iterrows():
    # Determine marker color based on launch outcome
    color = row['marker_color']
    
    # Create a marker for each launch with a popup label showing its outcome
    marker = folium.Marker(
        [row['Lat'], row['Long']],
        # Create an icon as a text label
        icon=folium.Icon(color=color, icon_color='white'),
        popup=row['class']
    )
    marker_cluster.add_child(marker)

# Add marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map

## TASK 3. Calculate the distances between a launch site to its proximities


Add a `MousePosition` on the map to get coordinate for a mouse over a point on the map.

In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [14]:
# Coordinate of CCAFS SLC-40
launch_site_lat = spacex_df.loc[spacex_df['Launch Site'] == 'CCAFS SLC-40', 'Lat'].iloc[0]
launch_site_lon = spacex_df.loc[spacex_df['Launch Site'] == 'CCAFS SLC-40', 'Long'].iloc[0]

# Find coordinate of the closest coastline
coastline_lat = 28.56395
coastline_lon = -80.56800

# Calculate distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [15]:
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)
site_map.add_child(coastline_marker)

# Draw a PolyLine between the launch site and the selected coastline point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

Draw a line betwee a launch site to its closest city, railway, highway, etc.

A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


Create a marker with distance to a closest city, railway, highway, etc.
Draw a line between the marker to the launch site

In [16]:
# Coordinate of CCAFS LC-40
launch_site_lat1 = spacex_df.loc[spacex_df['Launch Site'] == 'CCAFS LC-40', 'Lat'].iloc[0]
launch_site_lon1 = spacex_df.loc[spacex_df['Launch Site'] == 'CCAFS LC-40', 'Long'].iloc[0]

# Find coordinate of the closest railway
railway_lat = 28.57475
railway_lon = -80.58512

# Calculate distance between the launch site and the coastline
distance_railway = calculate_distance(launch_site_lat1, launch_site_lon1, railway_lat, railway_lon)

railway_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_railway),
    )
)
site_map.add_child(railway_marker)

# Draw a PolyLine between the launch site and the selected coastline point
lines = folium.PolyLine(locations=[[launch_site_lat1, launch_site_lon1], [railway_lat, railway_lon]], weight=1)
site_map.add_child(lines)

In [17]:
# Find coordinate of the closest highway
highway_lat = 28.56366
highway_lon = -80.57083

# Calculate distance between the launch site and the coastline
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

highway_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_highway),
    )
)
site_map.add_child(highway_marker)

# Draw a PolyLine between the launch site and the selected coastline point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]], weight=1)
site_map.add_child(lines)

In [18]:
# Coordinate of KSC LC-39A
launch_site_lat2 = spacex_df.loc[spacex_df['Launch Site'] == 'KSC LC-39A', 'Lat'].iloc[0]
launch_site_lon2 = spacex_df.loc[spacex_df['Launch Site'] == 'KSC LC-39A', 'Long'].iloc[0]

# Find coordinate of the closest city
city_lat = 28.61218
city_lon = -80.80805

# Calculate distance between the launch site and the coastline
distance_city = calculate_distance(launch_site_lat2, launch_site_lon2, city_lat, city_lon)

city_marker = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_city),
    )
)
site_map.add_child(city_marker)

# Draw a PolyLine between the launch site and the selected coastline point
lines = folium.PolyLine(locations=[[launch_site_lat2, launch_site_lon2], [city_lat, city_lon]], weight=1)
site_map.add_child(lines)

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?